In [210]:
import os
import openai
import shutil

In [211]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [212]:
!pip install Gitpython

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [213]:
from git import Repo

In [214]:
from pathlib import Path

In [215]:
#Just testing printing working directory

In [216]:
pwd

'C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io'

In [217]:
PATH_TO_BLOG_REPO = Path('C:\\Users\\david\\Desktop\\Python\\Courses\\Udemy\\OpenAI Python Bootcamp\\dawidahs.github.io\\.git')

In [218]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [219]:
PATH_TO_CONTENT = PATH_TO_BLOG/'content'

In [220]:
PATH_TO_CONTENT

WindowsPath('C:/Users/david/Desktop/Python/Courses/Udemy/OpenAI Python Bootcamp/dawidahs.github.io/content')

In [221]:
PATH_TO_CONTENT.mkdir(exist_ok=True,parents=True)

In [222]:
def update_blog(commit_message="Updates blog"):
    repo = Repo(PATH_TO_BLOG_REPO)
    repo.git.add(all=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [223]:
#random_text_string = 'smlkmssdlsdvdsdvsdmsdl'

In [224]:
#with open(PATH_TO_BLOG/'index.html', 'w') as f:
    #f.write(random_text_string)

In [225]:
 #Test push an update

In [226]:
#update_blog()

In [227]:
def create_new_blog(title, content, cover_image):
    cover_image = Path(cover_image)
    
    files = len(list(PATH_TO_CONTENT.glob('*.html')))
    new_title = f'{files+1}.html'
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        with open(path_to_new_content, 'w') as f:
            f.write('<DOCTYPE HTML>\n')
            f.write('<html>\n')
            f.write('<head>\n')
            f.write(f'<title>{title}</title>\n')
            f.write('</head>\n')
            
            f.write('<body>\n')
            f.write(f"<image src='{cover_image.name}' alt='Cover Image'>\n")
            f.write(f'<h1>{title}</h1>\n')
            f.write(content.replace("\n", "<br />\n"))
            f.write('</body>\n')
            
            f.write('</html>\n')
            print('Blog Created')
            return(path_to_new_content)
    else:
        raise FileExistsError('File already exists, please check you name.')

In [228]:
path_to_new_content = create_new_blog('Test Title','aklfakssadansdlasn','test.jpg')

Blog Created


In [229]:
from bs4 import BeautifulSoup as Soup

In [230]:
with open(PATH_TO_BLOG/'index.html') as index:
    soup = Soup(index.read())

In [231]:
soup

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Document</title>
</head>
<body>
<h1>My Blog Home Page</h1>
<a href="index.html">Home</a>
</body>
</html>

In [232]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get('href'))for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

In [233]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all('a')
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError('Link already exist!')
        
    link_to_new_blog = soup.new_tag('a', href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog_string = path_to_new_content.name.split('.')[0]
        last_link.insert_after(link_to_new_blog)
        
        with open(PATH_TO_BLOG/'index.html', 'w') as f:
            f.write(str(soup.prettify(formatter="html")))

In [234]:
write_to_index(path_to_new_content)

In [235]:
update_blog()